In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
sys.path.insert(0,'src/')

from collections import namedtuple

import torch 
from torch.utils.data import Dataset, DataLoader
import joblib
from torch import nn 
from sklearn.preprocessing import OneHotEncoder
import numpy as np


from architecture.reward import LearnedReward, StateDataset
from architecture.language_model import LanguageModel
from simulator.description_embedder import Description_embedder
from simulator.Environment import preprocess_raw_observation
from simulator.Items import ITEM_TYPE
from config import generate_params

params = generate_params(save_path=False)

In [4]:
# EpisodeRecord = namedtuple('EpisodeRecord', ('initial_state', 'final_state', 'instruction', 'reward'))
# episode_path = 'results/episodes_records.jbl'
# episodes = joblib.load(episode_path)

StateRecord = namedtuple('StateRecord', ('state', 'instruction', 'reward'))
state_path = 'results/state_records.jbl'
states = joblib.load(state_path)

In [5]:
description_embedder = Description_embedder(**params['env_params']['description_embedder_params'])

item_type_embedder = OneHotEncoder(sparse=False)
item_type_embedder.fit(np.array(ITEM_TYPE).reshape(-1, 1))

from functools import partial
transformer = partial(preprocess_raw_observation, description_embedder=description_embedder, item_type_embedder=item_type_embedder, raw_state_size=3, pytorch=True, device='cuda:1')

dts = StateDataset.from_files(state_path, raw_state_transformer=transformer)
train_dts, test_dts = dts.split(train_test_ratio = 0.7)
train_loader = DataLoader(train_dts, batch_size=3, shuffle=True)

In [103]:
language_model = LanguageModel(**params['language_model_params'])
reward = LearnedReward(context_model=params['model_params']['context_model'], language_model=language_model, reward_params=params['reward_model_params'])
reward.to('cuda:1')

LearnedReward(
  (context_net): FlatStateNet()
  (language_model): LanguageModel(
    (embedding_layer): Embedding(400000, 100)
    (lstm): LSTM(100, 100, bias=False, batch_first=True)
  )
  (reward_layer): Linear(in_features=212, out_features=1, bias=True)
)

In [8]:
next(iter(train_loader))

{'state': {'first plug': {'switch_binary': tensor([[-0.1448, -0.0486,  0.4747, -0.3888,  0.1440, -0.0708, -0.2636,  0.1610,
            -0.1533,  0.0830,  0.2066, -0.0049,  0.1361,  0.1452,  0.2833, -0.4874,
             0.1362,  0.0941, -0.2673, -0.1039,  0.3777, -0.1584,  0.5419,  0.0702,
             0.5689, -0.0347, -0.1799, -0.2112,  0.0585, -0.0346, -0.0213,  0.5968,
            -0.1230,  0.1413, -0.2211,  0.2017, -0.0221,  0.0932, -0.1351, -0.1498,
            -0.3616, -0.4191,  0.1478, -0.2925, -0.2135,  0.0494,  0.0854, -0.4467,
             0.1087, -0.5263, -0.2297,  0.2225, -0.0679,  1.2398, -0.1259, -2.5852,
            -0.1366, -0.2021,  1.8878,  0.6584, -0.1862,  0.5954, -0.1002,  0.4102,
             0.7728, -0.0163,  0.2757,  0.0341,  0.2244, -0.0811, -0.2381, -0.2915,
            -0.0548, -0.5484,  0.0988, -0.0371, -0.1928,  0.0544, -0.6491,  0.0474,
             0.7331, -0.0205, -0.4662,  0.2406, -1.1614, -0.1591,  0.1063,  0.1138,
            -0.0800, -0.4124, -0.210

In [93]:
train_dts[0]['state']['first plug']['switch_binary'].size()

torch.Size([112])

In [104]:
for i, batch in enumerate(train_loader):
    print(i)
    print(len(batch['instruction']))
    print(len(batch['reward']))
    print(batch['state']['first plug']['switch_binary'].size())
    print(reward(state = batch['state'], instructions=batch['instruction']).size())
    
    if i == 0:
        break

0
3
3
torch.Size([3, 112])


RuntimeError: Sizes of tensors must match except in dimension 0. Got 1 and 3

In [1]:
class StateDataset(Dataset):
    def __init__(self,)

In [7]:
from pprint import pprint
pprint(states[0])

StateRecord(state={'first plug': {'switch_binary': {'state': [1], 'description': 'Switch the power on and off.', 'item_type': 'switch'}}, 'first light bulb': {'color': {'state': [319, 15, 0], 'description': 'This channel supports full color control with hue, saturation and brightness values', 'item_type': 'color'}, 'color_temperature': {'state': [70], 'description': 'This channel supports adjusting the color temperature from cold (0%) to warm (100%)', 'item_type': 'dimmer'}}}, instruction='You turned on the first light bulb', reward=False)


In [2]:
def remove_key(state, key='embedding'):
    s = state.copy()
    try:
        s.pop(key)
    except KeyError:
        pass
    
    for k, v in s.values():
        if isinstance(v, dict):
            s[k] = remove_key(v, key)
    return s

In [ ]:
remove_key(episodes[1]['initial_state'])

In [14]:
e = episodes[0]

In [ ]:
e.final_state

In [6]:
def remove_key(state, key='embedding'):
    s = state.copy()
    if isinstance(s, dict):
        try:
            s.pop(key)
        except KeyError:
            pass
    
        for k, v in s.items():
            if isinstance(v, dict):
                s[k] = remove_key(v, key)
    return s

In [9]:
d = {
    'a': {
        'aa':{
            'aaa': 1,
            'embedding': 0
        },
        'embedding': 2
    },
    'b':{
        'embedding': 3
    },
    'embedding': 5
}

In [10]:
remove_key(d)

{'a': {'aa': {'aaa': 1}}, 'b': {}}

In [11]:
a = states[0]
a._asdict()

{'state': {'first plug': {'switch_binary': {'state': [1],
    'description': 'Switch the power on and off.',
    'item_type': 'switch'}},
  'first light bulb': {'color': {'state': [319, 15, 0],
    'description': 'This channel supports full color control with hue, saturation and brightness values',
    'item_type': 'color'},
   'color_temperature': {'state': [70],
    'description': 'This channel supports adjusting the color temperature from cold (0%) to warm (100%)',
    'item_type': 'dimmer'}}},
 'instruction': 'You turned on the first light bulb',
 'reward': False}

In [13]:
a._fields

('state', 'instruction', 'reward')

In [14]:
%timeit a._asdict()

600 ns ± 36.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [21]:
%timeit {k:a.__getattribute__(k) for k in a._fields}

670 ns ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
a.__getattribute__('state')

{'first plug': {'switch_binary': {'state': [1],
   'description': 'Switch the power on and off.',
   'item_type': 'switch'}},
 'first light bulb': {'color': {'state': [319, 15, 0],
   'description': 'This channel supports full color control with hue, saturation and brightness values',
   'item_type': 'color'},
  'color_temperature': {'state': [70],
   'description': 'This channel supports adjusting the color temperature from cold (0%) to warm (100%)',
   'item_type': 'dimmer'}}}